#### Etape 3.3 : Detection d'anomalies
- Identifier les pics de consommation anormaux (>3 ecarts-types)
- Detecter les periodes de sous-consommation suspectes (batiment ferme non declare)
- Reperer les batiments dont la consommation ne correspond pas a leur DPE
- Lister les batiments necessitant un audit energetique

**Livrables** :
- Notebook `08_detection_anomalies.ipynb`
- Liste des anomalies `output/anomalies_detectees.csv`
- Rapport de recommandations d'audit


In [5]:
import pandas as pd
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local") \
    .appName("08_detection_anomalies") \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

df = spark.read.parquet("../output/consommations_enrichies")
df = df.toPandas()

#- Identifier les pics de consommation anormaux (>3 ecarts-types)
df_pivot = df.pivot_table(
    values="consommation",
    index=["timestamp", "batiment_id"],
    columns="type_energie",
    aggfunc="sum",
    fill_value=0
).reset_index()

type_energies = ["eau", "gaz", "electricite"]

for col in type_energies:
    if col in df_pivot.columns:
        mean_val = df_pivot[col].mean()
        std_val = df_pivot[col].std()
        df_pivot["anomalie_" + col] = ((df_pivot[col] - mean_val).abs() > 3 * std_val)
        n_anomalies = df_pivot["anomalie_" + col].sum()
        print(f"{col} : {n_anomalies} pics anormaux")

# - Detecter les sous-consommation suspectes (batiment ferme non declare)
seuil_sous_consommation = 0.1  # kWh
for col in type_energies:
    if col in df_pivot.columns:
        df_pivot["sous_consommation_" + col] = (df_pivot[col] < seuil_sous_consommation)
        n_sous_consommations = df_pivot["sous_consommation_" + col].sum()
        print(f"{col} : {n_sous_consommations} sous-consommation suspectes")


# - Liste des anomalies `output/anomalies_detectees.csv`
anomalies = df_pivot[df_pivot.filter(regex="anomalie_|sous_consommation_").any(axis=1)]
anomalies.to_csv("/output/anomalies_detectees.csv", index=False)



Py4JJavaError: An error occurred while calling o60.parquet.
: java.lang.UnsatisfiedLinkError: 'boolean org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(java.lang.String, int)'
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Native Method)
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access(NativeIO.java:793)
	at org.apache.hadoop.fs.FileUtil.canRead(FileUtil.java:1249)
	at org.apache.hadoop.fs.FileUtil.list(FileUtil.java:1454)
	at org.apache.hadoop.fs.RawLocalFileSystem.listStatus(RawLocalFileSystem.java:601)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.fs.ChecksumFileSystem.listStatus(ChecksumFileSystem.java:761)
	at org.apache.spark.util.HadoopFSUtils$.listLeafFiles(HadoopFSUtils.scala:180)
	at org.apache.spark.util.HadoopFSUtils$.$anonfun$parallelListLeafFilesInternal$1(HadoopFSUtils.scala:95)
	at scala.collection.TraversableLike.$anonf

In [ ]:
# - Identifier les pics de consommation anormaux (>3 ecarts-types)